<a href="https://colab.research.google.com/github/konductor000/tg-GPT-2_bot/blob/main/bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install tensorflow-gpu==1.12.0
!pip3 install pyTelegramBotAPI

     |████████████████████████████████| 281.7MB 45kB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 3.1MB 51.6MB/s 
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.12.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
     |████████████████████████████████| 81kB 4.1MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-3.7.6-cp36-none-any.whl size=59261 sha256=83984c42fb0d92a3b9b5b808af8faf37de2c82283cff741ba39eb9fb76fdbb41
  Stored in directory: /root/.cache/pip/wheels/f6/66/6e/d42e8fcb446d2683b5afe0b23318f4bb58896bad26549c47b9
Successfully built pyTelegramBotAPI


In [22]:
# Use Tensorflow 1
%tensorflow_version 1.x

# Select GPT-2 model: 117M, 124M, 355M, 774M, 1558M

model_name = '117M'     # '117M' - smallest model. Set '1558M' for biggest 1.5B model

!git clone https://github.com/openai/gpt-2

%cd gpt-2

!pip3 install -r requirements.txt

# Download GPT-2 (selected model, for example 117M)
!python3 download_model.py $model_name



Cloning into 'gpt-2'...
remote: Enumerating objects: 233, done.
remote: Total 233 (delta 0), reused 0 (delta 0), pack-reused 233
Receiving objects: 100% (233/233), 4.38 MiB | 19.24 MiB/s, done.
Resolving deltas: 100% (123/123), done.
/content/gpt-2/src/gpt-2/src/gpt-2
Fetching checkpoint: 1.00kit [00:00, 791kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 1.89Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 666kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:16, 30.3Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 4.38Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 1.37Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 1.29Mit/s]                                                       


In [23]:
%cd src

/content/gpt-2/src/gpt-2/src/gpt-2/src


In [24]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder
import telebot


In [46]:
class GPT():


  def __init__(
      self,
      model_name,
      seed,
      length,
      temperature,
      top_k,
      models_dir
  ):
      models_dir = os.path.expanduser(os.path.expandvars(models_dir))


      self.enc = encoder.get_encoder(model_name, models_dir)
      hparams = model.default_hparams()
      with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
          hparams.override_from_dict(json.load(f))

      if length is None:
          length = hparams.n_ctx // 2
      elif length > hparams.n_ctx:
          raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

      g = tf.Graph() 
      with g.as_default():
      
        self.context = tf.placeholder(tf.int32, [1, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        self.output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=self.context,
            batch_size=1,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
      
      self.sess = tf.compat.v1.Session(graph=g) 
      saver.restore(self.sess, ckpt)


  def print_message(self, raw_text):
    context_tokens = self.enc.encode(raw_text)
    
    out = self.sess.run(self.output, feed_dict={
        self.context: [context_tokens for _ in range(1)]
    })[:, len(context_tokens):]
    for i in range(1):
        text = self.enc.decode(out[i])
        return text


In [47]:
gpt = GPT(
    '117M',
    None,
    100,
    1,
    40,
    '/content/gpt-2/models'
)

INFO:tensorflow:Restoring parameters from /content/gpt-2/models/117M/model.ckpt


In [48]:
print(gpt.print_message("hello"))

.com

Follow @brianwiles19

Share<|endoftext|>The U.S. Supreme Court has issued a ruling that says a federal law requiring that all drivers' licenses be renewed or suspended for at least a year violates the First Amendment rights of the driver.

United States v. Miller, 477 U.S. ___

This lawsuit alleges that a court-authorized search by the U.S. Marshal's Office of the Federal Bureau of Investigation led to the destruction


In [50]:


bot = telebot.TeleBot("1602318304:AAG0YvOX8MhX7nEWqmJjyVpq5EdQhgTnVLg")

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, f'Hi, {message.from_user.first_name}! I am a GPT-2 bot. Enter some text and you will receive a continue of your text.')


@bot.message_handler(content_types=['text'])
def send_text(message):
    text = message.text
    bot.send_message(message.chat.id, text + " " + gpt.print_message(text))

bot.polling(none_stop=True)

INFO:tensorflow:Restoring parameters from /content/gpt-2/models/117M/model.ckpt


RuntimeError: ignored

In [ ]:
!ls

encoder.py			    model.py
generate_unconditional_samples.py   sample.py
interactive_conditional_samples.py
